In [ ]:
import requests
import json
import urllib3
import warnings
import time

warnings.filterwarnings("ignore", category=urllib3.exceptions.InsecureRequestWarning)

host = "re-cluster1.ps-redislabs.org"
port = "9443"
username = "admin@rl.org"
password = "QFJ5GWf"

# URLs
url_db = f"https://{host}:{port}/v1/bdbs"
url_user = f"https://{host}:{port}/v1/users"
db_name = "v1-api-db"


In [ ]:
# Step 1: Delete users by UID if they already exist
user_emails = ["john.doe@example.com", "mike.smith@example.com", "cary.johnson@example.com"]
response_users = requests.get(
    url_user,
    auth=(username, password),
    headers={"Accept": "application/json"},
    verify=False
)

if response_users.status_code == 200:
    all_users = response_users.json()
    for email in user_emails:
        match = next((u for u in all_users if u.get("email") == email), None)
        if match:
            uid = match.get("uid")
            del_resp = requests.delete(
                f"{url_user}/{uid}",
                auth=(username, password),
                headers={"Accept": "application/json"},
                verify=False
            )
            if del_resp.status_code == 200:
                print(f"Deleted user {email} (uid={uid})")
            else:
                print(f"Failed to delete user {email} (uid={uid}) - {del_resp.status_code}")
        else:
            print(f"User {email} not found.")
else:
    print(f"Failed to fetch users for deletion - {response_users.status_code}")
    exit(1)


In [ ]:

# Step 2: Create the database
payload_db = {
    "name": db_name,
    "type": "redis",
    "memory_size": 2 * 1024 * 1024 * 1024
}
response_db = requests.post(
    url_db,
    auth=(username, password),
    headers={
        "Content-Type": "application/json",
        "Accept": "application/json"
    },
    data=json.dumps(payload_db),
    verify=False
)

if response_db.status_code == 200:
    db_uid = response_db.json().get("uid")
    print(f"✅ Database '{db_name}' created successfully.")
else:
    print(f"❌ Failed to create database '{db_name}': {response_db.status_code}")
    exit(1)


In [ ]:

# Step 3: Create users
users = [
    {"email": "john.doe@example.com", "name": "John Doe", "role": "db_viewer", "password": "asdf"},
    {"email": "mike.smith@example.com", "name": "Mike Smith", "role": "db_member", "password": "asdf"},
    {"email": "cary.johnson@example.com", "name": "Cary Johnson", "role": "admin", "password": "asdf"},
]

for user in users:
    resp = requests.post(
        url_user,
        auth=(username, password),
        headers={
        "Content-Type": "application/json",
        "Accept": "application/json"
        },
        data=json.dumps(user),
        verify=False
    )
    if resp.status_code == 200 or "uid" in resp_json:
        print(f"User {user['email']} created.")
    else:
        print(f"Failed to create user {user['email']} - {resp.status_code}")


In [ ]:
# STep 4: Fetch and display all users
response_fetch_users = requests.get(
    url_user,
    auth=(username, password),
    headers={
        "Accept": "application/json"
    },
    verify=False  # Skip cert check (for self-signed certs/dev only)
)
print("\nList of Users:")
if response_fetch_users.status_code == 200:
    users_list = response_fetch_users.json()
    for user in users_list:
        name = user.get("name")
        role = user.get("role")
        email = user.get("email")
        print(f"Name: {name}, Role: {role}, Email: {email}")
else:
    print(f"Failed to fetch users - {response_fetch_users.status_code}: {response_fetch_users.reason}")
    try:
        print(json.dumps(response_fetch_users.json(), indent=4))
    except Exception:
        print(response_fetch_users.text)

In [ ]:

# Step 5: Delete the database with retry
def is_db_ready(uid):
    r = requests.get(f"{url_db}/{uid}", auth=(username, password), verify=False)
    return r.status_code == 200 and r.json().get("status") == "active"

max_attempts = 5
for attempt in range(1, max_attempts + 1):
    if is_db_ready(db_uid):
        r = requests.delete(f"{url_db}/{db_uid}", auth=(username, password), verify=False)
        if r.status_code == 200:
            print(f"🗑️ Database '{db_name}' deleted successfully after {attempt} attempt(s).")
        else:
            print(f"❌ Failed to delete database '{db_name}': {r.status_code}")
        break
    time.sleep(5)
else:
    print(f"❌ Database '{db_name}' was not ready for deletion after max retries.")